In [ ]:
import os
import google.genai as genai
from dotenv import load_dotenv

load_dotenv(".env")

api_key = os.environ.get("GEMINI_API_KEY")
print("GEMINI_API_KEY found:", bool(api_key))
if api_key:
    print("GEMINI_API_KEY (partial):", api_key[:4] + "...")
else:
    print("Warning: GEMINI_API_KEY not found in environment")

client = None
if api_key:
    try:
        client = genai.Client(api_key=api_key)
    except Exception as e:
        print("Error creating client:", type(e).__name__, e)

print("--- MODELOS DISPONIBLES EN TU CUENTA ---")
if not client:
    print("No se creó cliente; revisa la API key y permisos.")
else:
    try:
        models_iter = client.models.list()
        models = list(models_iter)
        print("Número de modelos recibidos:", len(models))
        if not models:
            print("Lista vacía. Puede que tu cuenta no tenga modelos con esos permisos o haya un problema de permisos.")
        for i, m in enumerate(models, 1):
            name = getattr(m, "name", None)
            methods = getattr(m, "supported_generation_methods", None)
            print(f"Modelo #{i}: name={name}")
            print("  supported_generation_methods:", methods)
            # Mostrar una vista corta del objeto para diagnosticar
            try:
                print("  repr:", repr(m)[:300])
            except Exception:
                print("  (no se pudo repr el objeto)")
    except Exception as e:
        print("Error al listar modelos:", type(e).__name__, e)

print("--- FIN DIAGNÓSTICO ---")

GEMINI_API_KEY found: True
GEMINI_API_KEY (partial): AIza...
--- MODELOS DISPONIBLES EN TU CUENTA ---
--- MODELOS DISPONIBLES EN TU CUENTA ---
Número de modelos recibidos: 54
Modelo #1: name=models/embedding-gecko-001
  supported_generation_methods: None
  repr: Model(
  description='Obtain a distributed representation of a text.',
  display_name='Embedding Gecko',
  input_token_limit=1024,
  name='models/embedding-gecko-001',
  output_token_limit=1,
  supported_actions=[
    'embedText',
    'countTextTokens',
  ],
  tuned_model_info=TunedModelInfo(),
  ve
Modelo #2: name=models/gemini-2.5-flash
  supported_generation_methods: None
  repr: Model(
  description='Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025.',
  display_name='Gemini 2.5 Flash',
  input_token_limit=1048576,
  max_temperature=2.0,
  name='models/gemini-2.5-flash',
  output_token_limit=6553
Modelo #3: name=models/gemini-2.5-pro
  supported_


-- 1. Habilitar extensión de vectores
create extension if not exists vector;

-- 2. Borrar la tabla si existe (para empezar de cero)
drop table if exists documentos_dj;

-- 3. Crear la tabla NUEVA (Estructura correcta para tu Excel)
create table documentos_dj (
  id bigserial primary key,
  content text,
  metadata jsonb,
  embedding vector(768) -- Coincide con tu dimensión
);

-- 4. Crear el índice de búsqueda rápida
create index on documentos_dj using hnsw (embedding vector_cosine_ops);

-- 5. Habilitar permisos (CRUCIAL para que Python pueda escribir)
grant all on table documentos_dj to anon, authenticated, service_role;

create or replace function match_documentos (
  query_embedding vector(768),
  match_threshold float,
  match_count int
)
returns table (
  id bigint,
  content text,
  metadata jsonb,
  similarity float
)
language plpgsql
as $$
begin
  return query
  select
    documentos_dj.id,
    documentos_dj.content,
    documentos_dj.metadata,
    1 - (documentos_dj.embedding <=> query_embedding) as similarity
  from documents_dj
  where 1 - (documentos_dj.embedding <=> query_embedding) > match_threshold
  order by documentos_dj.embedding <=> query_embedding
  limit match_count;
end;
$$;

In [1]:
import os
import requests
from dotenv import load_dotenv

# Cargar variables desde .env
load_dotenv(".env")

# Obtener API Key
api_key = os.environ.get("DEEPSEEK_API_KEY")

if not api_key:
    raise ValueError("No se encontró DEEPSEEK_API_KEY en el archivo .env")

# Endpoint
url = "https://api.deepseek.com/models"

# Headers
headers = {
    "Accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# Petición GET
response = requests.get(url, headers=headers)

# Mostrar resultado
print("Status:", response.status_code)
print("Response JSON:")
print(response.json())


Status: 200
Response JSON:
{'object': 'list', 'data': [{'id': 'deepseek-chat', 'object': 'model', 'owned_by': 'deepseek'}, {'id': 'deepseek-reasoner', 'object': 'model', 'owned_by': 'deepseek'}]}
